AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [55]:
#  !pip install autogluon.timeseries

In [56]:
# Importar librerías
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [57]:
# Cargar datasets
df = pd.read_csv('sell-in.txt', sep='\t')
df_productos_predecir = pd.read_csv('product_id_apredecir201912.txt', sep='\t')

In [58]:
#Filter df to contain only products that are in df_productos_predecir
product_ids_to_predict = df_productos_predecir['product_id'].unique()
df = df[df['product_id'].isin(product_ids_to_predict)]

print(f"Original df shape after filtering: {df.shape}")
print(f"Unique products in df_productos_predecir: {len(product_ids_to_predict)}")
print(f"Unique products in filtered df: {df['product_id'].nunique()}")

Original df shape after filtering: (2293481, 7)
Unique products in df_productos_predecir: 780
Unique products in filtered df: 780


In [59]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [60]:
pivot_df = df.pivot_table(
    index=['customer_id', 'product_id'],
    columns='periodo',
    values='tn',
    aggfunc='sum'
).reset_index()

pivot_df.head()

periodo,customer_id,product_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,10001,20001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,10001,20002,87.64856,66.08396,75.09182,49.51494,122.40283,167.46470,156.15120,18.15133,...,220.19153,155.81927,264.55349,151.12081,103.12062,148.91108,213.36148,430.90803,547.87849,334.03714
2,10001,20003,100.21284,126.97776,114.52896,37.34640,76.66386,108.30456,87.14160,43.57080,...,125.49948,86.54509,74.71874,78.79703,105.81480,121.06458,101.61982,196.18531,135.69192,137.98717
3,10001,20004,21.73954,29.76246,42.54996,9.31694,8.33349,10.92153,15.01063,12.42259,...,25.94769,17.84712,27.99741,34.26047,16.04585,8.33349,20.57492,37.88891,27.58851,12.94020
4,10001,20005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.66966,1.72238,4.25654,3.20851,5.41195,2.51269,5.66966,7.98907,11.01719,7.66693


In [61]:
# Fill NaN values following the rule: keep NaN for values before the first non-null value in each row
def fill_nans_after_first_value(row):
    # Get the time series columns (excluding customer_id and product_id)
    time_columns = row.index[2:]  # Assuming first 2 columns are customer_id and product_id

    # Find the first non-null index
    first_non_null_idx = None
    for idx in time_columns:
        if pd.notna(row[idx]):
            first_non_null_idx = idx
            break

    # If no non-null value found, return the row as is
    if first_non_null_idx is None:
        return row

    # Fill NaN values with 0 only after the first non-null value
    first_non_null_position = time_columns.get_loc(first_non_null_idx)
    for i in range(first_non_null_position + 1, len(time_columns)):
        col = time_columns[i]
        if pd.isna(row[col]):
            row[col] = 0

    return row

# Apply the function to fill NaN values
pivot_df_filled = pivot_df.apply(fill_nans_after_first_value, axis=1)

In [62]:
# Convert pivot_df_filled back to long format
long_df = pivot_df_filled.melt(
    id_vars=['customer_id', 'product_id'],
    var_name='periodo',
    value_name='tn'
)

# Remove rows with NaN values if needed
long_df = long_df.dropna()

long_df.head()

,customer_id,product_id,periodo,tn
0,10001.0,20001.0,201701,99.43861
1,10001.0,20002.0,201701,87.64856
2,10001.0,20003.0,201701,100.21284
3,10001.0,20004.0,201701,21.73954
5,10001.0,20006.0,201701,29.17196


In [63]:
long_df['customer_id'] = long_df['customer_id'].astype(int)
long_df['product_id'] = long_df['product_id'].astype(int)
long_df['periodo'] = pd.to_datetime(long_df['periodo'], format='%Y%m')
long_df = long_df.rename(columns={'periodo': 'timestamp'})

long_df.head()

/tmp/ipython-input-63-3195971618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_df['customer_id'] = long_df['customer_id'].astype(int)
/tmp/ipython-input-63-3195971618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_df['product_id'] = long_df['product_id'].astype(int)


,customer_id,product_id,timestamp,tn
0,10001,20001,2017-01-01,99.43861
1,10001,20002,2017-01-01,87.64856
2,10001,20003,2017-01-01,100.21284
3,10001,20004,2017-01-01,21.73954
5,10001,20006,2017-01-01,29.17196


In [64]:
# Agregar tn total por periodo y producto
df_monthly_product = long_df.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [65]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [66]:
df_monthly_product.head()

,timestamp,product_id,tn,item_id
0,2017-01-01,20001,934.77222,20001
1,2017-01-01,20002,550.15707,20002
2,2017-01-01,20003,1063.45835,20003
3,2017-01-01,20004,555.91614,20004
4,2017-01-01,20005,494.27011,20005


In [67]:
# prompt: crea un nuevo df con los datos solamente hasta 201910

df_hasta_201910 = df_monthly_product[df_monthly_product['timestamp'].dt.strftime('%Y%m').astype(int) <= 201910]

print(f"Shape of df_hasta_201910: {df_hasta_201910.shape}")
df_hasta_201910.head()

Shape of df_hasta_201910: (20815, 4)


,timestamp,product_id,tn,item_id
0,2017-01-01,20001,934.77222,20001
1,2017-01-01,20002,550.15707,20002
2,2017-01-01,20003,1063.45835,20003
3,2017-01-01,20004,555.91614,20004
4,2017-01-01,20005,494.27011,20005


In [68]:
# --- Inserta este código en una nueva celda después de la celda [9] ---

# Hacemos una copia para mantener el dataframe original intacto
df_with_lags = df_monthly_product.copy()

# Es fundamental ordenar por producto (item_id) y fecha (timestamp)
# para que el cálculo de los lags sea correcto para cada serie individual.
df_with_lags = df_with_lags.sort_values(by=['item_id', 'timestamp'])

# Usamos el método shift() de pandas para crear los lags.
# El groupby('item_id') es CRUCIAL para asegurar que los lags se calculan
# dentro de cada serie de producto y no se mezclen datos entre productos.
print("Creando lags de 12 meses...")
for i in range(1, 13):
    df_with_lags[f'tn_lag_{i}'] = df_with_lags.groupby('item_id')['tn'].shift(i)

# Nota: El método shift() introducirá valores NaN al principio de cada serie
# (ej. los primeros 3 meses para el lag 3). Esto es normal.
# AutoGluon puede manejar estos NaNs en las covariables.

print("\nAsí se ven las primeras filas del DataFrame con los nuevos lags:")
# Mostramos un producto específico para ver los lags en acción
print(df_with_lags[df_with_lags['item_id'] == 20001].head())


# --- Ahora, modifica tu celda [11] para usar este nuevo DataFrame ---

# 4. Crear TimeSeriesDataFrame (usando el dataframe con lags)
# AutoGluon detectará automáticamente las columnas 'tn_lag_...' como 'past_covariates'
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_with_lags,       # <--- ¡Asegúrate de usar el nuevo DataFrame!
    id_column='item_id',
    timestamp_column='timestamp'
)

ts_data = ts_data.fill_missing_values()


Creando lags de 12 meses...

Así se ven las primeras filas del DataFrame con los nuevos lags:
      timestamp  product_id          tn  item_id    tn_lag_1    tn_lag_2  \
0    2017-01-01       20001   934.77222    20001         NaN         NaN   
496  2017-02-01       20001   798.01620    20001   934.77222         NaN   
996  2017-03-01       20001  1303.35771    20001   798.01620   934.77222   
1498 2017-04-01       20001  1069.96130    20001  1303.35771   798.01620   
2000 2017-05-01       20001  1502.20132    20001  1069.96130  1303.35771   

       tn_lag_3   tn_lag_4  tn_lag_5  tn_lag_6  tn_lag_7  tn_lag_8  tn_lag_9  \
0           NaN        NaN       NaN       NaN       NaN       NaN       NaN   
496         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
996         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
1498  934.77222        NaN       NaN       NaN       NaN       NaN       NaN   
2000  798.01620  934.77222       NaN       NaN   

In [69]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60*4)

Beginning AutoGluon training... Time limit = 14400s
AutoGluon will save models to '/content/AutogluonModels/ag-20250720_171939'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          12
GPU Count:          1
Memory Avail:       75.20 GB / 83.48 GB (90.1%)
Disk Space Avail:   190.93 GB / 235.68 GB (81.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 14400,
 'verbosity': 2}

Provided train_data has 22375 rows, 780 time series. Median time series l

In [70]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [71]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [72]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
# resultado = resultado[resultado['timestamp'] == '2019-12-01']
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [73]:
# 💾 7. Guardar archivo

resultado.head()

,product_id,tn
1,20001,1314.764250
3,20002,1051.869756
5,20003,691.152568
7,20004,509.508893
9,20005,485.841105


In [75]:
resultado.to_csv("predicciones_febrero2020_fecha_01_07-autogluon-lags-12-a.csv", index=False)

In [76]:
# Mostrar los mejores modelos del predictor
print("Mejores modelos entrenados:")
print(predictor.leaderboard())

Mejores modelos entrenados:
                           model  score_val  pred_time_val  fit_time_marginal  \
0               WeightedEnsemble  -0.172537       5.440319           2.990858   
1   ChronosFineTuned[bolt_small]  -0.181681       0.084370         146.963305   
2                       PatchTST  -0.182315       0.430523          70.836650   
3      TemporalFusionTransformer  -0.184861       0.469445         183.404109   
4                         DeepAR  -0.189664       0.439579         152.189668   
5     ChronosZeroShot[bolt_base]  -0.190380       1.204539           1.408631   
6                        AutoETS  -0.200780       2.943950           2.806828   
7          DynamicOptimizedTheta  -0.203529       0.642275           0.752617   
8               RecursiveTabular  -0.228409       0.050440           1.985401   
9                  SeasonalNaive  -0.230073       0.506835           0.561199   
10                          TiDE  -0.235729       0.922073         220.202015   


# Con este script, modificando un poco la parte anterior, calculamos el error rate de una predicción de diciembre de 2019 para ver si el modelo es consistente

In [ ]:
# prompt: calcula el error rate respecto a 201912 en df_monthly_product
# Obtener las ventas reales para 201912
ventas_reales_201912 = df_monthly_product[df_monthly_product['timestamp'] == '2019-12-01']

# Renombrar la columna 'tn' para evitar conflictos al hacer merge
ventas_reales_201912 = ventas_reales_201912.rename(columns={'tn': 'tn_real'})

# Hacer merge con las predicciones
# Asegurarse de que ambas columnas de 'product_id' sean del mismo tipo
resultado['product_id'] = resultado['product_id'].astype(int)
ventas_reales_201912['product_id'] = ventas_reales_201912['product_id'].astype(int)

df_comparacion = pd.merge(
    resultado,
    ventas_reales_201912[['product_id', 'tn_real']],
    on='product_id',
    how='left'  # Usamos left join para mantener todas las predicciones
)

# Rellenar los posibles NaNs en 'tn_real' (esto puede ocurrir si se predijo un producto
# que no tuvo ventas reales en 201912). Dependiendo de la métrica, podrías querer
# rellenar con 0 o manejarlo de otra forma. Para el cálculo de error, rellenar con 0 es común.
df_comparacion['tn_real'] = df_comparacion['tn_real'].fillna(0)

# Calcular el error absoluto
df_comparacion['error_absoluto'] = abs(df_comparacion['tn'] - df_comparacion['tn_real'])

# Calcular el MAPE (Mean Absolute Percentage Error)
# Evitar división por cero si tn_real es 0
# Se puede usar una versión simétrica o añadir un epsilon para estabilidad numérica.
# Aquí usamos una versión simple, evitando la división por cero explícitamente.
df_comparacion['error_porcentual'] = (df_comparacion['error_absoluto'] / df_comparacion['tn_real']).replace([float('inf'), -float('inf')], float('nan'))
df_comparacion['error_porcentual'] = df_comparacion['error_porcentual'].fillna(0) # Si tn_real es 0 y la predicción también es 0, el error es 0. Si tn_real es 0 y la predicción no es 0, el error es indefinido o grande, el MAPE simple no lo maneja bien. MAPE es más útil cuando no hay muchos ceros en los datos reales.

# Calcular el error total (sum_abs_error / sum_real)
total_error_absoluto = df_comparacion['error_absoluto'].sum()
total_real = df_comparacion['tn_real'].sum()

if total_real > 0:
    error_rate = (total_error_absoluto / total_real) * 100
else:
    error_rate = float('inf') # Si no hubo ventas reales, el error rate es infinito si hay alguna predicción diferente de 0.

print(f"\nComparación de predicciones vs realidad para 201912:")
print(df_comparacion.head())

print(f"\nError Absoluto Total para 201912: {total_error_absoluto:.2f}")
print(f"Ventas Reales Totales para 201912: {total_real:.2f}")
print(f"Error Rate (Sum Abs Error / Sum Real) para 201912: {error_rate:.2f}%")

# También podemos calcular el MAE (Mean Absolute Error)
mae = df_comparacion['error_absoluto'].mean()
print(f"MAE (Mean Absolute Error) para 201912: {mae:.2f}")

# Calcular el MAPE para productos donde tn_real > 0
mape_non_zero = df_comparacion[df_comparacion['tn_real'] > 0]['error_porcentual'].mean() * 100
print(f"MAPE (para productos con ventas reales > 0) para 201912: {mape_non_zero:.2f}%")
